In [ ]:
# add our stuff to the path
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# other stuff
import time
import warnings
warnings.simplefilter(action='ignore')
import pandas as pd

# import our stuff
from importlib import reload
from src import connect, preprocess, synthetic, model, matching

In [ ]:
# PARAMETERS

# info to gain access to database, IDIR restricted 
CRED_PATH = '../credentials.txt'

# where model is stored. requires credentials.txt for full path 
MODEL_BASE_PATH = 'Model/Q32'

# which tables to access
RESPONSE_TABLE = 'dbo.AQ32RACE'
RESULTS_TABLE = 'AQ32RACE_RESULTS'
MASTER_RESULTS_TABLE = 'dbo.AQ32RACE_RESULTS_CodingDone_Final'

# which column to use/create 
RESPONSE_COLUMN = 'aq32race_combined'
OUTPUT_COLUMNS = 'q32race_c'
N_COLUMNS = 16

# delimiter to send back with concatenated results
DELIMITER = 'μ' 

# threshold for accepting as a flagged category
THRESHOLD = 0.5

# upper and lower thresholds for flagging as a possible category
TENTATIVE_UPPER = 0.75
TENTATIVE_LOWER = 0.25

## Full Pipeline

1. Read in data from database (IDIR restricted)
2. Load in model (from LAN)
3. Preprocess data (code stored on GitHub)
4. Create predictions based on word scores
5. Re-incorprate multiple-choice responses
7. Pull final values that were actually used after the end of all modeling/QA/manual work was done from database
   
   * note that this whole workflow is required because of the way the data is stored in the database - no longer have access to the model results, only the final post model and manual coding results only for this question.
     
9. Create metrics to compare:

    * how much the model added
    * how much it differed from the final outputs that were used 

In [ ]:
# FULL MODEL PIPELINE

######################
#                    #
#    READ IN DATA    #
#                    #
######################
print(f'Reading in data... ', end = '')

connection = connect.create_connection(CRED_PATH)
df_open = connect.fetch_table(RESPONSE_TABLE, connection)
# responses that have been done already
#df_done = connect.fetch_table(MASTER_RESULTS_TABLE, connection)

print('Done.')

######################
#                    #
#    LOAD MODEL      #
#                    #
######################
print('Loading model from file... ', end = '')

clf, code_df_long = model.load_model(CRED_PATH, MODEL_BASE_PATH)
print('Done.')

######################
#                    #
#  PREPROCESS DATA   #
#                    #
######################
print('Preprocessing data... ', end = '')

# get cycle
# get new ids
#completed_ids = df_done.id.unique()
df = df_open.copy()#[~df_open.id.isin(completed_ids)].reset_index(drop=True)

# clean column
df.loc[:, RESPONSE_COLUMN] = df.apply(
    lambda x: x.aq32race.lower() if x.aq32race_cleaned == None or x.aq32race_cleaned=='105' else x.aq32race_cleaned.lower(), 
    axis=1
)

# fix spelling
df.loc[:, RESPONSE_COLUMN] = df[RESPONSE_COLUMN].apply(preprocess.correct_spelling)

# translate, only those that start with &
#df.loc[df.aq32race.str[0] == '&', RESPONSE_COLUMN] = df[df.aq32race.str[0] == '&'][RESPONSE_COLUMN].apply(lambda x: matching.get_translation(x, skip=False)[0])

print('Done.')

print('Creating model inputs... ', end = '')
# inputs to model
headers = list(preprocess.get_scores('test', code_df_long, as_df = True).col_id.values)
input_df = preprocess.get_scores_from_df(df, RESPONSE_COLUMN, code_df_long, headers=headers)
print('Done.')

# outputs of model (for selected responses to be included)
print('Creating selected response outputs... ', end = '')
output_df = preprocess.get_outputs_wide(df, RESPONSE_COLUMN, code_df_long, OUTPUT_COLUMNS, N_COLUMNS)
print('Done.')

######################
#                    #
#     RUN MODEL      #
#                    #
######################
print('Running model and extracting results... ')
results_df = model.produce_results(
    df, input_df, output_df, 
    clf,
    OUTPUT_COLUMNS,
    N_COLUMNS,
    threshold=THRESHOLD,
    tentative_lower = TENTATIVE_LOWER,
    tentative_upper = TENTATIVE_UPPER,
    delimiter = DELIMITER
)
print('\nDone.')

######################
#                    #
# GET FINAL RESULTS  #
#                    #
######################
print(f'Getting final results... ', end = '')
connection = connect.create_connection(CRED_PATH)
final_df = connect.fetch_table(MASTER_RESULTS_TABLE, connection)
print('Done')

In [ ]:
# some basic model metrics
(
    results_df[['id', 'match', 'original_matched', 'extra_categories', 'n_original_categories', 'n_model_categories']].astype('float')
    .assign(added_categories = lambda x: (x['extra_categories']>0)*1)
    .agg(
        {
            'id': 'count',
            'match': 'sum',
            'original_matched': 'mean',
            'extra_categories': 'mean',
            'n_model_categories': 'mean',
            'added_categories': 'sum'
        }
    )
)

In [ ]:
results_df[~pd.isnull(results_df.tentative_categories)].shape[0]

In [ ]:
# compare the set of final model categories to the final set of final categories... 
def make_set(row):
    n_max = 16
    full_set = set()
    for ii in range(1,n_max+1):
        col_name = f'q32race_c{ii:02d}'
        val = row[col_name]
        if val is not None:
            full_set.add(val)
    return full_set
            
final_df['final_set'] = final_df.apply(make_set, axis=1)
final_df['final_comment'] = final_df['aq32race']
results_df['model_set'] = results_df.apply(make_set, axis=1)
results_df['model_comment'] = results_df['aq32race_combined']

In [ ]:
test_df = final_df[['new_id', 'final_set', 'final_comment']].merge(results_df[['id', 'model_set', 'model_comment']], how='left', left_on='new_id', right_on='id')
test_df['matches'] = test_df.apply(lambda x: x.final_set == x.model_set, axis=1)

In [ ]:
print(f'Total:   {test_df.shape[0]:,}')
print(f'Matched: {test_df.matches.sum():,}')
print(f'Percent: {test_df.matches.sum()/test_df.shape[0]:0.0%}')